# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part1_data = "output_data/Data Frame.csv"
weather_df = pd.read_csv(part1_data)
weather_df = weather_df.drop(['Unnamed: 0'], axis = 1 )


weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sorong,-0.8833,131.2500,85.15,69,14,5.30,ID,1613104583
1,Bambous Virieux,-20.3428,57.7575,84.00,65,40,9.22,MU,1613104540
2,Hobart,-42.8794,147.3294,73.40,38,0,19.57,AU,1613104264
3,Vaini,-21.2000,-175.2000,86.00,79,20,5.75,TO,1613104508
4,Albany,42.6001,-73.9662,14.00,62,20,9.22,US,1613104511


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)


fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity=100,
                                 point_radius = 1) 
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Ideal Temperature 60F - 90F
#Ideal Humidity 30% - 50%
#Ideal Cloudiness 0-7
#Ideal Wind Speed 0 - 15

ideal_weather = weather_df.loc[(weather_df["Max Temp"] >= 60) &
                               (weather_df["Max Temp"]<= 90) &
                               (weather_df["Wind Speed"] <= 15) &
                               (weather_df["Humidity"] <= 50) &
                               (weather_df["Cloudiness"] <= 7 ),:
                              ]
# ideal_weather.head()

if len(ideal_weather) > 10:
    ideal_weather = ideal_weather
    
ideal_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,Wawa,9.9022,4.4192,73.26,25,0,1.72,NG,1613104993
51,Busselton,-33.6500,115.3333,75.15,41,0,3.69,AU,1613104508
157,Dūngarpur,23.8333,73.7167,78.24,18,0,7.90,IN,1613105011
234,Tlalchapa,18.4000,-100.4667,72.66,39,0,1.36,MX,1613105022
249,Kota,25.1833,75.8333,75.85,21,0,6.17,IN,1613105024


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df = pd.DataFrame(ideal_weather)
hotel_df["Hotel Name"] = ""
#hotel_df.head()

params = {"radius": 5000, "types": "lodging",
          "keyword": "Hotel", "key": g_key}

for index, row in hotel_df.iterrows():
    hotel_lat = row["Lat"]
    hotel_lng = row["Lng"]
    params["location"] = f"{hotel_lat},{hotel_lng}"
    
    hotel_data = requests.get(base_url, params = params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        print("Hotel found")
    except(IndexError, KeyError):
        print("Hotel not found")

hotel_df = hotel_df.dropna()
hotel_df.head()

Hotel not found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel found
Hotel not found


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
39,Wawa,9.9022,4.4192,73.26,25,0,1.72,NG,1613104993,
51,Busselton,-33.6500,115.3333,75.15,41,0,3.69,AU,1613104508,The Sebel Busselton
157,Dūngarpur,23.8333,73.7167,78.24,18,0,7.90,IN,1613105011,Hotel Raj Palace
234,Tlalchapa,18.4000,-100.4667,72.66,39,0,1.36,MX,1613105022,
249,Kota,25.1833,75.8333,75.85,21,0,6.17,IN,1613105024,WelcomHeritage Umed Bhawan Palace


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [8]:


#RAN OUT OF TIME. CANNOT THINK OF SOLUTION 



# gmaps.configure(api_key = g_key)

# location2 = hotel_df[["Lat", "Lng"]].astype(float)

# hotel_df = hotel_df.dropna()
# weight2 = hotel_df.astype(float)


# fig2 = gmaps.figure()

# hotel_layer = gmaps.heatmap_layer(location2, hotel_info)

# fig.add_layer(hotel_layer)
# fig2
# # Add marker layer ontop of heat map
# # Display figure